📚 This guide explains how to use YOLOv5 🚀 model ensembling during testing and inference for improved mAP and Recall. UPDATED 25 January 2022.

From https://www.sciencedirect.com/topics/computer-science/ensemble-modeling:

Ensemble modeling is a process where multiple diverse models are created to predict an outcome, either by using many different modeling algorithms or using different training data sets. The ensemble model then aggregates the prediction of each base model and results in once final prediction for the unseen data. The motivation for using ensemble models is to reduce the generalization error of the prediction. As long as the base models are diverse and independent, the prediction error of the model decreases when the ensemble approach is used. The approach seeks the wisdom of crowds in making a prediction. Even though the ensemble model has multiple base models within the model, it acts and performs as a single model.

#Before You Start

Clone repo and install requirements.txt in a Python>=3.7.0 environment, including PyTorch>=1.7. Models and datasets download automatically from the latest YOLOv5 release.

In [1]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
!pip install -r requirements.txt  # install

Cloning into 'yolov5'...
remote: Enumerating objects: 10878, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 10878 (delta 1), reused 7 (delta 0), pack-reused 10866
Receiving objects: 100% (10878/10878), 10.99 MiB | 24.73 MiB/s, done.
Resolving deltas: 100% (7510/7510), done.
/content/yolov5
     |████████████████████████████████| 596 kB 5.2 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


#Test Normally

Before ensembling we want to establish the baseline performance of a single model. This command tests YOLOv5x on COCO val2017 at image size 640 pixels. yolov5x.pt is the largest and most accurate model available. Other options are yolov5s.pt, yolov5m.pt and yolov5l.pt, or you own checkpoint from training a custom dataset ./weights/best.pt. For details on all available models please see our README table.

In [2]:
!python val.py --weights yolov5x.pt --data coco128.yaml --img 640 --half

val: data=/content/yolov5/data/coco128.yaml, weights=['yolov5x.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.0-236-gcba4303 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

100% 166M/166M [00:02<00:00, 82.7MB/s]

Fusing layers... 
Model Summary: 444 layers, 86705005 parameters, 0 gradients

Dataset not found, missing paths: ['/content/datasets/coco128/images/train2017']
100% 6.66M/6.66M [00:00<00:00, 76.2MB/s]
Dataset autodownload success, saved to /content/datasets

val: Scanning '/content/datasets/coco128/labels/train2017' images and labels...128 found, 0 missing, 2 empty, 0 corrupt: 100% 128/128 [00:00<00:00, 1098.14it/s]
val: New cache created: /content/datasets/coco128/labels/train2017.cache
               Class     Images     Labels         

#Ensemble Test

Multiple pretraind models may be ensembled togethor at test and inference time by simply appending extra models to the --weights argument in any existing val.py or detect.py command. This example tests an ensemble of 2 models togethor:

- YOLOv5x
- YOLOv5l6

In [3]:
!python val.py --weights yolov5x.pt yolov5l6.pt --data coco128.yaml --img 640 --half

val: data=/content/yolov5/data/coco128.yaml, weights=['yolov5x.pt', 'yolov5l6.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.0-236-gcba4303 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
Model Summary: 444 layers, 86705005 parameters, 0 gradients
100% 147M/147M [00:01<00:00, 118MB/s]

Fusing layers... 
Model Summary: 476 layers, 76726332 parameters, 0 gradients
Ensemble created with ['yolov5x.pt', 'yolov5l6.pt']

val: Scanning '/content/datasets/coco128/labels/train2017.cache' images and labels... 128 found, 0 missing, 2 empty, 0 corrupt: 100% 128/128 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 4/4 [00:29<00:00,  7.31s/it]
                 all        128        92

#Ensemble Inference
Append extra models to the --weights argument to run ensemble inference:

In [4]:
!python detect.py --weights yolov5x.pt yolov5l6.pt --img 640 --source data/images

detect: weights=['yolov5x.pt', 'yolov5l6.pt'], source=data/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-236-gcba4303 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
Model Summary: 444 layers, 86705005 parameters, 0 gradients
Fusing layers... 
Model Summary: 476 layers, 76726332 parameters, 0 gradients
Ensemble created with ['yolov5x.pt', 'yolov5l6.pt']

image 1/2 /content/yolov5/data/images/bus.jpg: 640x512 4 persons, 1 bus, Done. (0.306s)
image 2/2 /content/yolov5/data/images/zidane.jpg: 384x640 3 persons, 3 ties, Done. (0.240s)
Speed: 0.6ms pre-process, 273.2ms inference, 3.8ms NMS per image at shape (1, 3, 640, 6